<h1> Musikjahr </h1>

This Data Science project aims to analyze a Spotify user's music trends in the previous year and recommends new artists based on those music trends.

Install libraries related to API

In [1]:
%pip install python-dotenv
%pip install requests

Install libraries related to Data Science tasks

In [2]:
%pip install seaborn

Install library related to Spotify OAuth2

In [3]:
from dotenv import load_dotenv
import os
import base64
import requests
import urllib.parse
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer

Matplotlib is building the font cache; this may take a moment.


In [4]:
load_dotenv()

True

In [5]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = "http://localhost:3001"
scope = "user-top-read"

In [6]:
# Returns the authorization header required to query the API
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [7]:
def get_auth_code():
    auth_url = "https://accounts.spotify.com/authorize"
    
    params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "user-top-read",
    }

    auth_url = f"{auth_url}?{urllib.parse.urlencode(params)}"
    print(f"Please Authorize access by clicking on: {auth_url}")

    auth_code = input("Enter the code in the generated URL: ")
    
    return auth_code


In [8]:
# Returns new refresh token
def exchange_auth_code_for_refresh_token(auth_code):
    token_url = "https://accounts.spotify.com/api/token"
    
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri,
        "client_id": client_id,
        "client_secret": client_secret
        }

    result = requests.post(token_url, data=data)
    json_result = json.loads(result.content)
    refresh_token = json_result['refresh_token']
    
    return refresh_token

In [9]:
# Returns a new access token upon every execution
def refresh_access_token(refresh_token):
    token_url = "https://accounts.spotify.com/api/token"
    
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": client_id,
        "client_secret": client_secret
        }

    result = requests.post(token_url, data=data)
    json_result = json.loads(result.content)
    new_access_token = json_result['access_token']

    return new_access_token

In [11]:
auth_code = get_auth_code()
print(auth_code)

Please Authorize access by clicking on: https://accounts.spotify.com/authorize?client_id=9e3ab84ec3634133b2710bc672715e09&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3001&scope=user-top-read
<PyodideFuture pending cb=[WebLoop._decrement_in_progress()]>


Enter the code in the generated URL:  AQA-8xpdn99148JP6CJlJJyrps2tXtPQvwz_Y4zh-m7aBHONvQNbRkSY5N85pxAYlre7yP4hQfybFMSoSNXQIHx1euIVePdhXch9UJuFDAzHaZ2aMIpk5fXHtTIjkyYI7ItVdExnLoFVHFHodh20bLBkDfW-mzWp5ZNbXsgMvAPsYHZYZP2YqA


In [12]:
refresh_token = exchange_auth_code_for_refresh_token(auth_code.result())

In [106]:
def get_top_tracks(refresh_token, time_period, amount):
    api_url = "https://api.spotify.com/v1/me/top/tracks"
    
    params = {
        "limit": amount,
        "time_range": str(time_period),
        "offset": 0
    }
    
    result = requests.get(api_url, headers=get_auth_header(refresh_access_token(refresh_token)), params=params)
    data = json.loads(result.content)

    songs_dict={"Song_Title": [], "Artists": []} # Add Popularity
    song_data = []
    artists_data = []

    
    for song in data['items']:
        song_data.append(song['name'])

    for song in data['items']:
        artists_data_list = song['album']['artists']
        if(len(artists_data_list) > 1):
            
            artists_for_song = []
            for artists in artists_data_list:
                artists_for_song.append(artists['name'])
            artists_data.append(artists_for_song)
        else:
            for artists in artists_data_list:
                artists_data.append(artists['name'])

    
    for song in song_data:
        songs_dict['Song_Title'].append(song)

        
    for artist in artists_data:
        songs_dict['Artists'].append(artist)
            
    return songs_dict


In [107]:
get_top_tracks(refresh_token, "short_term", 5) # Last 4 weeks

{'Song_Title': ['Snow On Tha Bluff',
  'Rain in Cuba',
  'You’re Gonna Go Far',
  'Winter Wonderland',
  'Stick Season'],
 'Artists': ['J. Cole', 'Joe Hertz', 'Noah Kahan', 'J. Cole', 'Noah Kahan']}

In [108]:
get_top_tracks(refresh_token, "medium_term", 5) # Last 6 months

{'Song_Title': ['baby pink (feat. Eem Triplin)',
  '3 Nights',
  "Don't Go Yet",
  'Jailhouse Rock',
  'Paint The Town Blue (from the series Arcane League of Legends)'],
 'Artists': ['Camila Cabello',
  'Dominic Fike',
  'Camila Cabello',
  'Elvis Presley',
  ['Ashnikko', 'Arcane', 'League of Legends']]}

In [117]:
get_top_tracks(refresh_token, "long_term", 5) # Last 12 months

{'Song_Title': ['Bothmer Schloss',
  'baby pink (feat. Eem Triplin)',
  '3 Nights',
  'Feel It - From The Original Series “Invincible”',
  'Young And Beautiful'],
 'Artists': ['Marie Bothmer',
  'Camila Cabello',
  'Dominic Fike',
  'd4vd',
  'Various Artists']}